In [1]:
import numpy as np
import pandas as pd
from IPython.display import display

import torch

from pytorch_pretrained_bert.tokenization import BertTokenizer
from pytorch_pretrained_bert.modeling import BertForMaskedLM

In [19]:
model_id = 'bert-base-cased'
tokenizer = BertTokenizer.from_pretrained(model_id, do_lower_case=False)
model = BertForMaskedLM.from_pretrained(model_id)
model.eval()
pass

11/26/2018 13:32:57 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt from cache at /Users/victor/.pytorch_pretrained_bert/5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1
11/26/2018 13:32:57 - INFO - pytorch_pretrained_bert.modeling -   loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased.tar.gz from cache at /Users/victor/.pytorch_pretrained_bert/a803ce83ca27fecf74c355673c434e51c265fb8a3e0e57ac62a80e38ba98d384.681017f415dfb33ec8d0e04fe51a619f3f01532ecea04edbfd48c5d160550d9c
11/26/2018 13:32:57 - INFO - pytorch_pretrained_bert.modeling -   extracting archive file /Users/victor/.pytorch_pretrained_bert/a803ce83ca27fecf74c355673c434e51c265fb8a3e0e57ac62a80e38ba98d384.681017f415dfb33ec8d0e04fe51a619f3f01532ecea04edbfd48c5d160550d9c to temp dir /var/folders/vd/6zpy6cdx28538

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1)
            )
          )
          (intermediate): BertIntermediate(
       

In [87]:
# the tokenizer can represent any word by spelling it, if necessary
all('##'+c in tokenizer.vocab for c in 'abcdefghijklmnopqrstuvwxyz')

True

In [88]:
tokenizer.tokenize('xkcd')

['x', '##k', '##c', '##d']

In [91]:
# but it's not all that smart
tokenizer.tokenize('foodservice'), tokenizer.tokenize('food service')

(['foods', '##er', '##vice'], ['food', 'service'])

In [90]:
# and vocab is missing many proper nouns
tokenizer.tokenize('Alcatraz')

['Al', '##cat', '##raz']

In [85]:
np.random.choice(list(tokenizer.vocab.keys()), size=200)

array(['##ather', 'Zambia', '白', '##glass', '##umour', 'shade', 'Mat',
       '##esign', 'ne', 'deep', 'Product', 'mage', 'Ron', 'Yukon',
       'Arabian', 'detachment', '##garh', 'nods', 'ties', 'Flags',
       'plaza', 'polo', 'announcer', 'Stratford', '₃', 'selfish',
       'Windows', 'Romantic', 'Libby', '##ppard', 'ABS', 'say', 'Hooper',
       '##kai', 'rhythm', '##ienne', 'Subsequent', '##EE', 'ň', '@',
       'Lower', 'wooded', 'engines', 'table', 'Romano', 'museum',
       'forest', 'Arena', '##phones', 'Gone', '##ֹ', 'DL', 'cooler',
       'revival', 'Decision', '##umatic', 'Joel', '##ska', 'cm', 'viola',
       '1990', '##watch', 'た', 'incomplete', 'terrorist', 'filmmaker',
       '##llah', 'survivors', 'Visiting', 'Below', '##child',
       'Gloucester', 'Sai', 'shift', 'Visiting', 'doctoral', 'rabbit',
       '##lsive', 'ACT', 'think', '##rr', 'Triangle', 'cried', 'queen',
       'topping', 'rational', '##bit', 'men', 'sense', 'employs',
       'scoring', 'Custom', 'Indie'

In [275]:
# text = tokenizer.tokenize('this is a simple test sentence.')
# text = tokenizer.tokenize('certain methods handle self-referential sentences poorly.')
# text = tokenizer.tokenize('who was jim henson?')
# text = tokenizer.tokenize('Is there life on Mars?')
# text = tokenizer.tokenize('The Occupation of Alcatraz was an occupation of Alcatraz Island by 89 American Indians and supporters, led by Richard Oakes, LaNada Means, and others.')
text = tokenizer.tokenize('Beam search uses breadth-first search to build its search tree.')
# response = tokenizer.tokenize('jim henson was a puppeteer')
# response = ['[MASK]']*5 + ['!']
# response = list(np.random.choice([t for t in tokenizer.vocab if not t.startswith('[unused')], size=5)) + ['.']
# response = tokenizer.tokenize('so is this one.')
# response = tokenizer.tokenize('however, some may perform better than others.')
# response = tokenizer.tokenize('They chose the name Indians of All Tribes and John Trudell was the spokesperson.')
response = tokenizer.tokenize('At each level of the tree, it generates all successors of the states at the current level, sorting them in increasing order of heuristic cost')
response = ['[MASK]']*(len(response)-1) + ['.']
# response[0] = '[MASK]'
# response[3] = '[MASK]'
# response[5] = '[MASK]'

In [284]:
# tokens = torch.tensor(tokenizer.convert_tokens_to_ids(text+response))
tokens = torch.tensor(tokenizer.convert_tokens_to_ids(['[CLS]']+text+['[SEP]']+response))
# segments = torch.tensor([0]*len(text) + [1]*len(response))
segments = torch.tensor([0]*(len(text)+2) + [1]*len(response))

In [285]:
token_prior = torch.tensor([1.0]*len(tokenizer.vocab))
# token_prior[1012] *= 0
# token_prior[[1000, 1006, 1007, 1010]] *= 0.1

In [ ]:
display(' '.join(tokenizer.convert_ids_to_tokens(np.array(tokens))))

In [299]:
def display_text(t):
    display(' '.join(tokenizer.convert_ids_to_tokens(np.array(t))).replace(' ##', ''))

In [302]:
# deterministically fix the most confident [MASK] token one at a time (resembles beam search with size 1?)
pred_tokens = tokens.clone()
idxs_to_predict = [i for i,t in enumerate(tokens) if t==103]
while True:
    display_text(pred_tokens[len(text)+2:])
    
    if not len(idxs_to_predict): break
    
    pred = (
        (model(pred_tokens.unsqueeze(dim=0), segments.unsqueeze(dim=0))[0]*token_prior)
        .index_select(0, torch.LongTensor(idxs_to_predict))
        )
    
    p, i = torch.max(pred, dim=1)
    most_confident_idx = torch.argmax(p).item()
    token_id = i[most_confident_idx].item()
    pred_tokens[idxs_to_predict[most_confident_idx]] = token_id
    del idxs_to_predict[most_confident_idx]

'[MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] .'

'[MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] . [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] .'

'[MASK] [MASK] [MASK] [MASK] - [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] . [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] .'

'[MASK] [MASK] - [MASK] - [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] . [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] .'

'[MASK] [MASK] - to - [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] . [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] .'

...

'The end - to - end search tree is [MASK] [MASK] . For [MASK] search , search , search , and search . See [MASK] search tree .'

'The end - to - end search tree is called [MASK] . For [MASK] search , search , search , and search . See [MASK] search tree .'

'The end - to - end search tree is called search . For [MASK] search , search , search , and search . See [MASK] search tree .'

'The end - to - end search tree is called search . For [MASK] search , search , search , and search . See also search tree .'

'The end - to - end search tree is called search . For example search , search , search , and search . See also search tree .'

In [303]:
# iteratively sample from naive joint distribution 
# does not resemble data distribution but generates pretty concrete poetry
display_text(pred_tokens[1:len(text)+1])
pred_tokens = tokens.clone()
for _ in range(32):
#     display(' '.join(tokenizer.convert_ids_to_tokens(np.array(pred_tokens))).replace(' ##', ''))   
    pred_tokens[len(text)+2:] = model(pred_tokens.unsqueeze(dim=0), segments.unsqueeze(dim=0))[0, len(text)+2:].exp().multinomial(1).flatten()
    display_text(pred_tokens[len(text)+2:])


'Beam search uses breadth - first search to build its search tree .'

'Federal offer - - = his - , , out . a - = , - - . ,dcher couldN carder andp .'

"' offer - - = his - , , out . a - = , - - . ,d , andN carder andp ."

"' end - - = - - , , out . a - = , - - . ,d , and ' carder andp ."

"' end - - = - - , , out . a - = , - - . ,d , . ' 'er andp ."

...

"' add - - = - - , , * . a - = , - - . , , , . and ' . ap ."

"' add - - = - - , , * . a - = , - - . , , , . and ' . ap ."

"' add - - = - - , , * . a - = , - - . , , , . , ' . ap ."

"' add - - = - - , , * . a - = , - - . , , , . , ' . ap ."

"' add - - = - - , , * . a - = , - - . , , , . , ' . ap ."

In [304]:
# iteratively sample tokens from softmax, left to right
pred_tokens = tokens.clone()
for i in range(len(text)+2, len(tokens)):
    pred_tokens[i] = model(pred_tokens.unsqueeze(dim=0), segments.unsqueeze(dim=0))[0, i].exp().multinomial(1).item()
    display_text(pred_tokens[len(text)+2:])


'intrinsic [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] .'

'intrinsic Horn [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] .'

'intrinsic Horn String [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] .'

'intrinsic Horn Strings [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] .'

'intrinsic Horn Strings < [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] .'

...

"intrinsic Horn Strings < + < < < < < < < < < < < < < > Francis ' s top [MASK] [MASK] [MASK] ."

"intrinsic Horn Strings < + < < < < < < < < < < < < < > Francis ' s top search [MASK] [MASK] ."

"intrinsic Horn Strings < + < < < < < < < < < < < < < > Francis ' s top search group [MASK] ."

"intrinsic Horn Strings < + < < < < < < < < < < < < < > Francis ' s top search group [SEP] ."

"intrinsic Horn Strings < + < < < < < < < < < < < < < > Francis ' s top search group [SEP] ."

In [289]:
# # iteratively sample from second sentence, one random position at a time
# pred_tokens = tokens.clone()
# it = 0
# while it < 4*len(response) or 103 in pred_tokens:
#     it+=1
#     i = np.random.randint(len(text)+2, len(tokens))
#     pred_tokens[i] = model(pred_tokens.unsqueeze(dim=0), segments.unsqueeze(dim=0))[0, i].exp().multinomial(1).item()
#     display_text(pred_tokens[len(text)+2:])

In [307]:
# iteratively sample one position at a time, but choose i to have maximum entropy
# usually terminates pretty fast but can apparently get stuck with some low-entropy distributions over remaining mask tokens
temp = 1
pred_tokens = tokens.clone()
k = len(text)+2
it=0
while 103 in pred_tokens and it < 64:# or it < 2*len(response) :
    it+=1
    pred = model(pred_tokens.unsqueeze(dim=0), segments.unsqueeze(dim=0))[0, k:].exp()
    pred /= pred.sum(dim=1, keepdim=True)
    ent = -(pred.log() * pred).sum(dim=1)
#     display(ent)
    i = ent.argmax()
    pred_tokens[i+k] = pred[i].pow(1/temp).multinomial(1).item()
    display_text(pred_tokens[len(text)+2:])


'[MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] ? .'

'[MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] & ? .'

'[MASK]ick [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] & ? .'

'[MASK]ick , [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] & ? .'

'[MASK]ick , x [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] & ? .'

...

'/ , , x / , [MASK] ) , " ; ? ? = " ( " > ) ; ? = < " , / ? .'

'/ , , x / , [MASK] ) , " ; ? ? = " ( " > ) ; ? = < " , / ? .'

'/ , , x / , [MASK] ) , " ; ? ? = " ( " > ) ; ? = < " , / ? .'

'/ , , x / , [MASK] / , " ; ? ? = " ( " > ) ; ? = < " , / ? .'

'/ , , x / , > / , " ; ? ? = " ( " > ) ; ? = < " , / ? .'

In [308]:
# iteratively fix one [MASK] token at a time, treating the flattened logits as one big softmax
temp = 1
pred_tokens = tokens.clone()
idxs_to_predict = [i for i,t in enumerate(tokens) if t==103]
# idxs_to_predict = list(range(2+len(text), len(tokens)))
while True:
    display_text(pred_tokens[len(text)+2:])
    
    if not len(idxs_to_predict): break
    
    pred = (
        (model(pred_tokens.unsqueeze(dim=0), segments.unsqueeze(dim=0))[0]*token_prior)
        .index_select(0, torch.LongTensor(idxs_to_predict))
        ).exp()
    pred = (pred/pred.sum(dim=1, keepdim=True)).pow(1/temp).view(-1)
    
    ij = pred.multinomial(1).item()
    i = ij//len(tokenizer.vocab)
    pred_tokens[idxs_to_predict[i]] = ij%len(tokenizer.vocab)
    del idxs_to_predict[i]

'[MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] .'

'[MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] are [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] .'

'[MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] & [MASK] [MASK] are [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] .'

'[MASK] [MASK] [MASK] [MASK] beat [MASK] [MASK] [MASK] [MASK] & [MASK] [MASK] are [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] .'

'[MASK] [MASK] [MASK] [MASK] beat [MASK] [MASK] [MASK] [MASK] & [MASK] [MASK] are [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] / [MASK] .'

...

'# [MASK] × live beatmember , K & [MASK]E are listed on his / [MASK] their [MASK] . # × universal i / m .'

'# [MASK] × live beatmember , K & SE are listed on his / [MASK] their [MASK] . # × universal i / m .'

'# # × live beatmember , K & SE are listed on his / [MASK] their [MASK] . # × universal i / m .'

'# # × live beatmember , K & SE are listed on his / her their [MASK] . # × universal i / m .'

'# # × live beatmember , K & SE are listed on his / her their album . # × universal i / m .'

In [ ]:
#idea: linguistic erosion stalactites. iteratively delete, replace, shorten tokens
# for max entropy position, zero prob of current token then sample
# delete min entropy position
# zero prob of all tokens longer than current token in position, then sample